In [1]:
import ast 

In [ ]:
class VASPStyleParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.params = {
            'PROCAR_PATH': None,
            'ISPIN': 1,
            'ORBITAL_INFO': None,  #Required
            'SCALE': 1,
            'TRANSPARENCY':70,
            'EFERMI': None,
            'TITLE': 'Orbital projected Band Structure',
            'FIGSIZEX': 10,
            'FIGSIZEY': 6,
            'PLOT_OPTION': 0, #0 for scatter 1 for parametric
            'COLOR_SCHEME': None,  # Required validation later
            'YMIN': -5.0,
            'YMAX': 5.0,
            'LINEWIDTH': 1.0,
            'DPI': 300,
            'SAVEAS': 'orbband.jpg',
            
        }
        self._parse()
        self._validate()
        self._apply_default_color_scheme()
    def _parse(self):
        with open(self.filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue

                # Remove inline comments , only keep stuff before first inline comment 
                
                if '#' in line:
                    line = line.split('#', 1)[0].strip()
                #after splitting the comment also if there are invalid inputs without proper key value assignment
                #it will catch them
                if not line or '=' not in line:
                    continue

                key, value = map(str.strip, line.split('=', 1))#str strip to remove white spaces before after equal to sign
                
                key = key.upper()#if user enters lowercase

                if key not in self.params:
                    raise ValueError(f"Unknown config key: {key}. Please enter proper config key.")

                # PARSIng loGiC
                if key =='ORBITAL_INFO':
                    try:
                        self.params[key] = ast.literal_eval(value)
                        if not isinstance(self.params[key], list):
                            raise ValueError
                    except Exception:
                        raise ValueError("ORBITAL_INFO must have a valid nested python list type structure.")
                
                
                elif key == 'COLOR_SCHEME':
                    if self.params['PLOT_OPTION'] == 0:
                        try:
                            val = ast.literal_eval(value)

                            # single integer case
                            if isinstance(val, int):
                                self.params[key] = val

                            #second case: list of strings (color names)
                            elif isinstance(val, list) and all(isinstance(v, str) for v in val):
                                self.params[key] = val

                            else:
                                raise ValueError
                        except Exception:
                            raise ValueError(
                                "COLOR_SCHEME must be either a single integer or a list of color strings when PLOT_OPTION is 0."
                            )
                    else:
                        # When PLOT_OPTION is 1, it must be a matplotlib colormap string
                        self.params[key] = value.strip()
                elif key in ['ISPIN', 'PLOT_OPTION','SCALE']:
                    self.params[key] = int(value)
                elif key in ['YMIN', 'YMAX', 'FIGSIZEX', 'FIGSIZEY','DPI','LINEWIDTH','TRANSPARENCY']:
                    try:
                        self.params[key] = float(value)
                    except ValueError:
                        raise ValueError(f"{key} must be a numerical value.")
                elif key == 'EFERMI':
                    try:
                        self.params[key] = float(value)
                    except ValueError:
                        raise ValueError("EFERMI must be a float if provided.")
                elif key == 'SAVEAS':
                    val = value.strip()
                    if not (val.lower().endswith('.jpg') or val.lower().endswith('.png')):
                        raise ValueError("SAVEAS must end with .jpg or .png")
                    self.params[key] = val
                else:
                    self.params[key] = value
    
    def _validate(self):
        if self.params['ORBITAL_INFO'] is None:
            raise ValueError("ORBITAL_INFO is Required and cannot be None.")

        if self.params['PLOT_OPTION'] not in [0, 1]:
            raise ValueError("PLOT_OPTION must be 0 or 1.")

        if self.params['COLOR_SCHEME'] is None:
            raise ValueError("COLOR_SCHEME must be defined.")

        if self.params['PLOT_OPTION'] == 1:
            if not isinstance(self.params['COLOR_SCHEME'], str):
                raise ValueError("When PLOT_OPTION is 1, COLOR_SCHEME must be a valid colormap name.")
    
    def _apply_default_color_scheme(self):
        if self.params['COLOR_SCHEME'] is None:
            self.params['COLOR_SCHEME'] = 0 if self.params['PLOT_OPTION'] == 0 else "plasma"
    #utility functions to be called later in the code for easy calling
    def get(self, key):
        return self.params.get(key.upper())

    def as_dict(self):
        return self.params.copy()
        

In [ ]:
class BandPlot:
    def __init__(
        self,
        procar_path,
        ispin=1,
        orbital_info=None,
        efermi=0.0,
        title="Band Structure",
        figsize=(10, 6),
        plot_option=0,
        color_scheme=None,
        ymin=-5.0,
        ymax=5.0,
    ):
        self.procar_path = procar_path
        self.ispin = ispin
        self.orbital_info = orbital_info
        self.efermi = efermi
        self.title = title
        self.figsize = figsize
        self.plot_option = plot_option
        self.color_scheme = color_scheme
        self.ymin = ymin
        self.ymax = ymax

    def plot(self):
        # your plotting logic
        pass

cli.py

In [ ]:
from orbibands.vasp_style_parser import VaspStyleConfigParser
from orbibands.bandplot import BandPlot

def main():
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--config', required=True)
    args = parser.parse_args()

    config = VaspStyleConfigParser(args.config)
    cfg = config.as_dict()

    band_plot = BandPlot(
        procar_path=cfg['PROCAR_PATH'],
        ispin=cfg['ISPIN'],
        orbital_info=cfg['ORBITAL INFO'],
        efermi=cfg['EFERMI'],
        title=cfg['TITLE'],
        figsize=(cfg['FIGSIZEX'], cfg['FIGSIZEY']),
        plot_option=cfg['PLOT_OPTION'],
        color_scheme=cfg['COLOR_SCHEME'],
        ymin=cfg['YMIN'],
        ymax=cfg['YMAX'],
    )

    band_plot.plot()

python cli.py --config orbibands.in
And the attributes in BandPlot will be automatically initialized, with type checking, defaults, and validation handled by your config parser.

📌 Bonus: Add a from_config() Classmethod (Optional)
You can optionally make the BandPlot class even cleaner by giving it a constructor helper:

In [3]:
@classmethod
def from_vasp_config(cls, config_path):
    from orbibands.vasp_style_parser import VaspStyleConfigParser
    cfg = VaspStyleConfigParser(config_path).as_dict()
    return cls(
        procar_path=cfg['PROCAR_PATH'],
        ispin=cfg['ISPIN'],
        orbital_info=cfg['ORBITAL INFO'],
        efermi=cfg['EFERMI'],
        title=cfg['TITLE'],
        figsize=(cfg['FIGSIZEX'], cfg['FIGSIZEY']),
        plot_option=cfg['PLOT_OPTION'],
        color_scheme=cfg['COLOR_SCHEME'],
        ymin=cfg['YMIN'],
        ymax=cfg['YMAX'],
    )

In [ ]:
band_plot = BandPlot.from_vasp_config(args.config)
band_plot.plot()